# t-academy-pytorch-lecture2

# Variable & Automatic Gradient

## 1. Import Required Libraries

In [1]:
import torch
from torch.autograd import Variable

## 2. Tensor vs Variable

### 1) Declaration

In [2]:
x_tensor = torch.Tensor(3, 4)
x_tensor

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [3]:
x_variable = Variable(x_tensor)
x_variable

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

### 2) Variables of a Variable

In [4]:
# .data -> wrapped tensor
x_variable.data

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [6]:
# .grad -> gradient of the variable
print(x_variable.grad) # 그라디언트 값이 없으니 None

None


In [8]:
# .requires_grad -> whether variable requires gradient
print(x_variable.requires_grad)

x_variable = Variable(x_tensor, requires_grad=True) # 그래디언트 계산 하라는 명령
x_variable.requires_grad

False


True

In [10]:
# .volatile -> inference mode with minimal memory usage
# sinlge voltile variable makes whole graph not requiring gradient

x_variable = Variable(x_tensor, volatile=True)
x_variable.grad, x_variable.requires_grad, x_variable.volatile # volatile 없어졌대요... 아래 워닝 읽어봐요.

C:\Users\Wyatt\anaconda3\envs\wyatt_torch\lib\site-packages\ipykernel_launcher.py:4: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  after removing the cwd from sys.path.
C:\Users\Wyatt\anaconda3\envs\wyatt_torch\lib\site-packages\ipykernel_launcher.py:5: UserWarning: volatile was removed (Variable.volatile is always False)
  """


(None, False, False)

## 3. Graph & Variables

In [14]:
# create graph

x = Variable(torch.FloatTensor(3, 4), requires_grad=True)
y = x**2 + 4*x
z = 2*y + 3

x.requires_grad, y.requires_grad, z.requires_grad

(True, True, True)

In [15]:
# .backward(gradient, retain_graph, create_graph, retain_variables)
# compute gradient of current variable w, r, t graph leaves

loss = torch.FloatTensor(3, 4)
z.backward(loss)

print(x.grad)
y.grad, z.grad

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


C:\Users\Wyatt\anaconda3\envs\wyatt_torch\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  


(None, None)

# Linear Regression vs Neural Quadratic

In [16]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

### 2.Generate Data

In [17]:
num_data = 1000
num_epoch = 1000

noise = init.normal(torch.FloatTensor(num_data, 1), std=1)
x = init.uniform(torch.Tensor(num_data, 1), -10, 10)

y = 2*x + 3
y_noise = y+noise

C:\Users\Wyatt\anaconda3\envs\wyatt_torch\lib\site-packages\ipykernel_launcher.py:4: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  after removing the cwd from sys.path.
C:\Users\Wyatt\anaconda3\envs\wyatt_torch\lib\site-packages\ipykernel_launcher.py:5: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  """


### 3. Model & Optimizer

In [28]:
# Linear model
model = nn.Linear(1, 1)
output = model(Variable(x))

loss_func = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [34]:
# fully connected model with 4 hidden layer

model = nn.Sequential(
        nn.Linear(1, 6),
        nn.ReLU(),
        nn.Linear(6, 10),
        nn.ReLU(),
        nn.Linear(10, 6),
        nn.ReLU(),
        nn.Linear(6, 1),
        )

loss_func = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

### 4. Train

In [35]:
# train
loss_arr = []

label = Variable(y_noise)
for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(Variable(x))
    
    loss = loss_func(output, label)
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(loss)
        
    loss_arr.append(loss.data.numpy())

tensor(10.2469, grad_fn=<L1LossBackward>)
tensor(10.2336, grad_fn=<L1LossBackward>)
tensor(10.2215, grad_fn=<L1LossBackward>)
tensor(10.2103, grad_fn=<L1LossBackward>)
tensor(10.2001, grad_fn=<L1LossBackward>)
tensor(10.1907, grad_fn=<L1LossBackward>)
tensor(10.1820, grad_fn=<L1LossBackward>)
tensor(10.1738, grad_fn=<L1LossBackward>)
tensor(10.1661, grad_fn=<L1LossBackward>)
tensor(10.1589, grad_fn=<L1LossBackward>)
tensor(10.1520, grad_fn=<L1LossBackward>)
tensor(10.1455, grad_fn=<L1LossBackward>)
tensor(10.1392, grad_fn=<L1LossBackward>)
tensor(10.1331, grad_fn=<L1LossBackward>)
tensor(10.1278, grad_fn=<L1LossBackward>)
tensor(10.1230, grad_fn=<L1LossBackward>)
tensor(10.1186, grad_fn=<L1LossBackward>)
tensor(10.1144, grad_fn=<L1LossBackward>)
tensor(10.1104, grad_fn=<L1LossBackward>)
tensor(10.1064, grad_fn=<L1LossBackward>)
tensor(10.1024, grad_fn=<L1LossBackward>)
tensor(10.0984, grad_fn=<L1LossBackward>)
tensor(10.0944, grad_fn=<L1LossBackward>)
tensor(10.0904, grad_fn=<L1LossBac

### 5. Check Trained Parameters

In [27]:
param_list = list(model.parameters())
print(param_list[0].data, param_list[1].data)

tensor([[2.0038]]) tensor([3.0359])
